<a href="https://www.kaggle.com/code/hsw1212/heartdisease-classification-use-grid-search-and-rf?scriptVersionId=139283624" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-classification-dataset/Heart Attack.csv


In [2]:
# Read csv data
data = pd.read_csv("/kaggle/input/heart-disease-classification-dataset/Heart Attack.csv")
data

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,64,1,66,160,83,160.0,1.80,0.012,negative
1,21,1,94,98,46,296.0,6.75,1.060,positive
2,55,1,64,160,77,270.0,1.99,0.003,negative
3,64,1,70,120,55,270.0,13.87,0.122,positive
4,55,1,64,112,65,300.0,1.08,0.003,negative
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,negative
1315,66,1,84,125,55,149.0,1.33,0.172,positive
1316,45,1,85,168,104,96.0,1.24,4.250,positive
1317,54,1,58,117,68,443.0,5.80,0.359,positive


In [3]:
# Check null information in csv file
data.info()
print("<<<<<<<<<<<<<The number of null data>>>>>>>>>>>>>")
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1319 non-null   int64  
 1   gender         1319 non-null   int64  
 2   impluse        1319 non-null   int64  
 3   pressurehight  1319 non-null   int64  
 4   pressurelow    1319 non-null   int64  
 5   glucose        1319 non-null   float64
 6   kcm            1319 non-null   float64
 7   troponin       1319 non-null   float64
 8   class          1319 non-null   object 
dtypes: float64(3), int64(5), object(1)
memory usage: 92.9+ KB
<<<<<<<<<<<<<The number of null data>>>>>>>>>>>>>


age              0
gender           0
impluse          0
pressurehight    0
pressurelow      0
glucose          0
kcm              0
troponin         0
class            0
dtype: int64

In [4]:
# Check mean, std, min, max, 25%, 50% and 75%.
data.describe()

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin
count,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000
mean,56.191812,0.659591,78.336619,127.170584,72.269143,146.634344,15.274306,0.360942
std,13.647315,0.474027,51.630270,26.122720,14.033924,74.923045,46.327083,1.154568
min,14.000000,0.000000,20.000000,42.000000,38.000000,35.000000,0.321000,0.001000
25%,47.000000,0.000000,64.000000,110.000000,62.000000,98.000000,1.655000,0.006000
50%,58.000000,1.000000,74.000000,124.000000,72.000000,116.000000,2.850000,0.014000
75%,65.000000,1.000000,85.000000,143.000000,81.000000,169.500000,5.805000,0.085500
max,103.000000,1.000000,1111.000000,223.000000,154.000000,541.000000,300.000000,10.300000


In [5]:
# Train data and test data
X_data = data.drop('class', axis=1)
data['class'] = data['class'].map({'negative': 0, 'positive': 1})
y_data = data['class']

from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X_data,y_data,test_size=0.2, random_state=100,shuffle=True)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


((1055, 8), (264, 8), (1055,), (264,))

In [6]:
# Use RandomForestClassifier and GradientBoostingClassifier
# Use Grid Search to find the optimised hyperparameters
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid for Random Forest
param_grid_rf = {
    'classifier__n_estimators': [50, 150, 250],
    'classifier__max_depth': [3, 5, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Hyperparameter grid for Gradient Boosting
param_grid_gb = {
    'classifier__n_estimators': [50, 150, 250],
    'classifier__learning_rate': [0.01, 0.1, 1],
    'classifier__max_depth': [3, 5, 10],
}

# Pipelines for the classifiers
pipeline_rf = Pipeline([
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline_gb = Pipeline([
    ('classifier', GradientBoostingClassifier(random_state=42))
])

In [7]:
# Grid search for Random Forest
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(x_train, y_train)

# Grid search for Gradient Boosting
grid_search_gb = GridSearchCV(pipeline_gb, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(x_train, y_train)

# Best models and scores
best_model_rf, best_score_rf = grid_search_rf.best_estimator_, grid_search_rf.best_score_
best_model_gb, best_score_gb = grid_search_gb.best_estimator_, grid_search_gb.best_score_

best_model_rf, best_score_rf, best_model_gb, best_score_gb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(Pipeline(steps=[('classifier',
                  RandomForestClassifier(max_depth=5, min_samples_split=5,
                                         n_estimators=250, random_state=42))]),
 0.9876777251184834,
 Pipeline(steps=[('classifier',
                  GradientBoostingClassifier(learning_rate=0.01, max_depth=5,
                                             n_estimators=150,
                                             random_state=42))]),
 0.9876777251184834)

In [8]:
from sklearn.metrics import accuracy_score

# Test accuracy
# Make predictions on the test set using the Random Forest model
y_pred_rf = best_model_rf.predict(x_test)

# Make predictions on the test set using the Gradient Boosting model
y_pred_gb = best_model_gb.predict(x_test)

# Calculate accuracy scores
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_gb = accuracy_score(y_test, y_pred_gb)

print(f"RF accuracy: {accuracy_rf*100:.3f}, GB accuracy: {accuracy_gb*100:.3f}")

RF accuracy: 98.485, GB accuracy: 98.106


In [9]:
# Get feature importances from the Random Forest model
importances_rf = best_model_rf.named_steps['classifier'].feature_importances_

# Get feature importances from the Gradient Boosting model
importances_gb = best_model_gb.named_steps['classifier'].feature_importances_

# Convert to DataFrame for easier visualization
df_importances = pd.DataFrame({
    'Feature': data.drop('class', axis=1).columns,
    'Importance_RF': importances_rf,
    'Importance_GB': importances_gb,
})

df_importances.sort_values(by='Importance_RF', ascending=False)

,Feature,Importance_RF,Importance_GB
7,troponin,0.576089,0.603757
6,kcm,0.275173,0.369295
0,age,0.072956,0.004835
5,glucose,0.020201,0.001296
3,pressurehight,0.019448,0.004812
4,pressurelow,0.014799,0.000920
2,impluse,0.014256,0.000675
1,gender,0.007079,0.014409
